## Simplest version of the transportation model

Next steps
- Convert emissions/distance into emissions/time (as a function of avg speed, which is a function of congestion) - fully endogenous model
    - MOSTLY DONE - have emission rate over time, need to integrate to get total emissions
- Try to model U-shaped curve with something like a (x-40)**4 + 1...
    - NOT STARTED
- replace Greenshield's model with arctan function
    - DONE
- rewrite functions using matrix notation
    - IN PROGRESS

What's missing (from the XPPAUT version)?
- power function (for tapering off near 0)
- urban sponge

In [9]:
using Plots
using LinearAlgebra
using LaTeXStrings
using Roots
using DifferentialEquations
using Interpolations
using RecursiveArrayTools

Function for calculating populations N (of each patch and corridor) at next time step. Demo case for two patches with two connecting corridors. Final version will use matrix notation.

In [2]:
function traffic_flow_demo!(dN, N, p, t)
    #=
        Returns (change in place)
            - dN: array-like, size (6,)
                    population values at next time step
        Arguments
            - dN: array-like, size (6,)
                    population values at next time step
            - N: array-like, size (6,)
                    population values at current time step
                    N = [P¹, P², C¹₁₂, C¹₂₁, C²₁₂, C²₂₁]
            - p: array-like, size (6,)
                    parameters
                    p = [α₁, α₂, β₁, β₂, p1_out, p2_out]
    =#
    P¹ = N[1]      # population in patch 1
    P² = N[2]      # population in patch 2
    C¹₁₂ = N[3]    # population in corridor 1, lane from patch 1 to patch 2
    C¹₂₁ = N[4]    # population in corridor 1, lane from patch 2 to patch 1
    C²₁₂ = N[5]    # population in corridor 2, lane from patch 1 to patch 2
    C²₂₁ = N[6]    # population in corridor 2, lane from patch 2 to patch 1

    α₁ = p[1]       # tolerance for congestion, originators in patch 1
    α₂ = p[2]       # tolerance for congestion, originators in patch 2
    β₁ = p[3]       # inverse road capacity, corridor 1
    β₂ = p[4]       # inverse road capacity, corridor 2
    p1_out = p[5]   # overall demand for leaving P1
    p2_out = p[6]   # overall demand for leaving P2

    # Fluxes
    F₁ᶜ¹ = p1_out * exp(-β₁ * α₁ * C¹₁₂) * P¹   # flux from patch 1 into corridor 1 (implicitly heading towards patch 2)
    F₂ᶜ¹ = p2_out * exp(-β₁ * α₂ * C¹₂₁) * P²   # flux from patch 2 into corridor 1 (implicitly heading towards patch 1)
    F₁ᶜ² = p1_out * exp(-β₂ * α₁ * C²₁₂) * P¹   # flux from patch 1 into corridor 2 (implicitly heading towards patch 2)
    F₂ᶜ² = p2_out * exp(-β₂ * α₂ * C²₂₁) * P²   # flux from patch 2 into corridor 2 (implicitly heading towards patch 1)

    Fc₁¹ = exp(-β₁ * C¹₂₁) * C¹₂₁     # flux from corridor 1 into patch 1 (implicitly lane from p2 to p1)
    Fc₁² = exp(-β₁ * C¹₁₂) * C¹₁₂     # flux from corridor 1 into patch 2 (implicitly lane from p1 to p2)
    Fc₂¹ = exp(-β₂ * C²₂₁) * C²₂₁     # flux from corridor 2 into patch 1 (implicitly lane from p2 to p1)
    Fc₂² = exp(-β₂ * C²₁₂) * C²₁₂     # flux from corridor 2 into patch 2 (implicitly lane from p1 to p2)
    
    # Net changes to state variables
    dN[1] = Fc₁¹ + Fc₂¹ - F₁ᶜ¹ - F₁ᶜ² # change in population Patch 1
    dN[2] = Fc₁² + Fc₂² - F₂ᶜ¹ - F₂ᶜ² # change in population Patch 2
    dN[3] = F₁ᶜ¹ - Fc₁² # change in population Corridor 1 lane from p1 to p2
    dN[4] = F₂ᶜ¹ - Fc₁¹ # change in population Corridor 1 lane from p2 to p1
    dN[5] = F₁ᶜ² - Fc₂² # change in population Corridor 2 lane from p1 to p2
    dN[6] = F₂ᶜ² - Fc₂¹ # change in population Corridor 2 lane from p2 to p1
end
# Where to put conservation law??

traffic_flow_demo! (generic function with 1 method)

Run diff eqs for 100 time steps, with initial conditions P¹=0, P²=C¹₁₂=C¹₂₁=C²₁₂=C²₂₁=0, and parameters given below. Plot outputs for each population.

In [3]:
# Choose parameters
α₁ = 1
α₂ = 1
β₁ = 10
β₂ = 60
p1_out = 1
p2_out = 0
p = [α₁, α₂, β₁, β₂, p1_out, p2_out]

# Solve diff eq
N0 = [1; 0.; 0; 0; 0; 0]                              # set initial conditions: [P¹, P², C¹₁₂, C¹₂₁, C²₁₂, C²₂₁]
tspan = (0.0, 100.0)                                  # set time span
prob = ODEProblem(traffic_flow_demo!, N0, tspan, p)   # create problem (system of diff eqs)
sol = solve(prob)                                     # solve problem

# Plot each population over time 
plot(sol, idxs = (0,1), xlabel="time", ylabel="population", label="patch 1")
plot!(sol, idxs = (0,2), xlabel="time", ylabel="population", label="patch 2")
plot!(sol, idxs = (0,3), xlabel="time", ylabel="population", label="C1 (1 to 2), β₁=$β₁")
plot!(sol, idxs = (0,5), xlabel="time", ylabel="population", label="C2 (1 to 2), β₂=$β₂")
plot!(sol, idxs = (0,4), xlabel="time", ylabel="population", label="C1 (2 to 1), β₁=$β₁")
plt = plot!(sol, idxs = (0,6), xlabel="time", ylabel="population", label="C2 (2 to 1), β₂=$β₂")
savefig(plt, "traffic_flows_1.png")
display(plt)

# Plot phase space
#plot(sol, idxs = (1,2), xlabel="P1", ylabel="P2")

LoadError: UndefVarError: `ODEProblem` not defined

### Functions to convert traffic densities to average speeds

In [4]:
# Calculate speeds from densities
v_f = 90             # free-flow velocity, 90 km/hr, same for C1 and C2
C1_jam = 1 / β₁      # jam density for C1 (causes avg speed = 0)
C2_jam = 1 / β₂      # jam density for C2 (causes avg speed = 0)
C1_half = C1_jam / 2 # threshold density for C1 (causes avg speed = 1/2 free-flow speed)
C2_half = C2_jam / 2 # threshold density for C2 (causes avg speed = 1/2 free-flow speed)

function calc_space_mean_speed_alternative(v_f, C, C_half; a=1)
    #=
        Returns:
            - u_s: float 
                average speed for vehicles in a given traffic flow. If negative, return 0.
        Arguments:
            - v_f: float
                free-flow velocity
            - C: float
                vehicle density (in corridor C)
            - C_half:
                threshold value of vehicle density, where u_s = 1/2 * v_f
    =#
    u_s = - (v_f / pi) * atan(a*(C - C_half)) + (v_f / 2)
    return u_s > 0 ? u_s : 0
end

# NOT USED - Greenshield's model
function calc_space_mean_speed_greenshields(v_f, k, k_jam)
    u_s = v_f - (k/k_jam)*v_f
    return u_s > 0 ? u_s : 0
end


calc_space_mean_speed_greenshields (generic function with 1 method)

### Functions to convert average speeds to emission rates

In [5]:
# Make a U-shaped curve using data from the California paper
start = 5 
my_step = 5
stop = 100
mph_to_kmh = 1.60934
speed_arr = collect(start:my_step:stop) * mph_to_kmh # convert mph to kmh
emissions_arr = [1200, 950, 700, 500, 425, 350, 325, 310, 309, 308, 308, 308, 309, 320, 330, 350, 375, 400, 450, 550] * mph_to_kmh
plot(speed_arr, emissions_arr)

# Interpolate: emissions as a function of speed
interp_fn = linear_interpolation(speed_arr, emissions_arr, extrapolation_bc=Line())

# Calculate emissions from a given array of speeds
function calc_emissions_from_speed(vehicle_pop_arr, my_speed_arr, interp_fn)
    #=
        Returns:
            - emissions: array (dim 1) 
                emission rates (g/km) for whole traffic volume (all vehicles) at each
                time step
        Arguments:
            - vehicle_pop_arr: array (dim 1) of vehicle population densities at 
              each time step
            - my_speed_arr: array (dim 1) of avg vehicle speeds at each time step
            - interp_fn: function (interpolated) relating speeds to emissions
    =#
    interpolated_emission_per_vehicle = interp_fn(my_speed_arr)
    emissions = interpolated_emission_per_vehicle .* my_speed_arr .* vehicle_pop_arr
    return emissions
end

# NOT USED - Calculate flow volume
function calc_flow(vehicle_pop_arr, v_f, k_jam)
    q = v_f .* vehicle_pop_arr - (v_f / k_jam) .* vehicle_pop_arr.^2
    return q
end

LoadError: UndefVarError: `plot` not defined

### Calculate Expected Emissions for travel on C1 and C2

In [6]:
pop_C1 = sol[3, :]
C1_speeds = calc_space_mean_speed_alternative.(v_f, pop_C1, C1_half)
C1_emissions = calc_emissions_from_speed(pop_C1, C1_speeds, interp_fn)
C1_flow = calc_flow(pop_C1, v_f, C1_jam)

plt1 = plot(sol, idxs=(0,3), xlabel="t", ylabel="population", label="C1", title="population C1 (β₁ = $β₁)")
plt1 = plot!(plt1, [0,100], repeat([C1_jam], 2), label="C_jam")
plt1 = plot!(plt1, [0,100], repeat([C1_half], 2), label="C_half")
plt2 = plot(sol.t, C1_speeds, xlabel="t", ylabel="v (km/h)", label="v", title="average speeds (assume v_f=90)")
plt3 = plot(sol.t, C1_emissions, xlabel="t", ylabel="CO2 (G / hour)", label="CO2", title="average emissions")
#plt4 = plot(sol.t, C1_flow, xlabel="t", ylabel="q (vehicles / hour)", label="q", title="traffic flow")

plt = plot(plt1, plt2, plt3, layout = grid(3,1, heights=(1/3,1/3,1/3)), size=(600,600))

savefig(plt, "congestion_to_emissions_C1.png")
display(plt)

LoadError: UndefVarError: `sol` not defined

In [7]:
# Same for C2
pop_C2 = sol[5, :]
C2_speeds = calc_space_mean_speed_alternative.(v_f, pop_C2, C2_half)
C2_emissions = calc_emissions_from_speed(pop_C2, C2_speeds, interp_fn)
C2_flow = calc_flow(pop_C2, v_f, C2_jam)

plt1 = plot(sol, idxs=(0,5), xlabel="t", ylabel="population", label="C2", title="population C2 (β₂ = $β₂)")
plt1 = plot!(plt1, [0,100], repeat([C2_jam], 2), label="C_jam")
plt1 = plot!(plt1, [0,100], repeat([C2_half], 2), label="C_half")
plt2 = plot(sol.t, C2_speeds, xlabel="t", ylabel="v (km/h)", label="v", title="average speeds (assume v_f=90)")
plt3 = plot(sol.t, C2_emissions, xlabel="t", ylabel="CO2 (G / hour)", label="CO2", title="average emissions")
#plt4 = plot(sol.t, C2_flow, xlabel="t", ylabel="q (vehicles / hour)", label="q", title="traffic flow")

plt = plot(plt1, plt2, plt3, layout = grid(3,1, heights=(1/3,1/3,1/3)), size=(600,600))

savefig(plt, "congestion_to_emissions_C2.png")
display(plt)

LoadError: UndefVarError: `sol` not defined

## WIP - Matrix form

In [ ]:
function traffic_flow!(dN, N, p, t)
    #=
        Returns (change in place)
            - dN: VectorOfArrays, (P, C)
                    population values at next time step
        Arguments
            - dN: VectorOfArrays, (P, C)
                    population values at next time step
            - N: VectorOfArrays, (P, C)
                    population values at current time step
                    N = [P¹, P², C¹₁₂, C¹₂₁, C²₁₂, C²₂₁]
            - p: VectorOfArrays, size (3, ...)
                    parameters
                    p = [αᵢ, βₖ, pi_out]
    =#

    P = N.x[1]       # array for populations of patches
    C = N.x[2]       # VectorOfArrays for populations in corridors, heading from patch i to patch j

    display(P)
    display(C)

    n = length(P)  # total number of patches
    K = length(C)  # maximum number of connective corridors between patches. Start with K=1.

    αᵢ = p[1]       # tolerance for congestion, originators in patch i
    βₖ = p[2]       # inverse road capacity, corridor k, heading from patch i to patch j
    pi_out = p[3]   # overall demand for leaching patch i

    # Fluxes
    empty_matrix = Matrix{Float64}(undef, n, n)
    Fₚᶜᵏ = VectorOfArray(empty_matrix) # empty array to hold fluxes from patch i into corridor k, heading towards patch j
    Fₖᵖ = VectorOfArray(empty_matrix)  # empty array to hold fluxes from corridor k into patch j (with an origin of patch i)

    for i in 1:n        # from patch i
        for j in 1:n    # to patch j
                Fₚᶜᵏ[i,j] = [pi_out[i] .* exp(-βₖ .* αᵢ[i] .* C[i,j]) .* P[i]] # flux from patch i into corridor k heading in the j direction
                Fₖᵖ[i,j] = exp(-βₖ .* C[i,j]) .* C[i,j]                # flux from corridor k (heading from patch i to patch j) into patch j
        end
    end
    
    # Net changes to state variables
    for i in 1:n
        for j in 1:n
                dN[1,i] = Fₖᵖ[:,i] .- Fₚᶜᵏ[i] # change in population in patch i
                dN[2,i,j] = Fₚᶜᵏ[i,j] .- Fₖᵖ[i,j] # change in population in corridor k going from patch i to patch j
        end
    end
end

# Solve diff eq
P0 = [0.9,0]
C0 = ArrayPartition([0.25,0.15], [0.2,0.4])
N0 = ArrayPartition(P0, C0)

α = [1,1]
βₖ = [60, 10]
pi_out = [1, 0]
p = VectorOfArray([α, βₖ, pi_out])

display(N0)
display(p)

println("EXPERIMENT")
i = 2; j = 2; k=2
println(N0[i])
println(N0.x[i])
#println(N0[j,i])
println(N0.x[i].x[j])
println(N0.x[i].x[j][1])
#println(N0[:,i])
#println(N0[j])
#println(N0[k,i,j])
println("langth: ", length(N0.x[i]))
#tspan = (0.0, 100.0)                                  # set time span
#prob = ODEProblem(traffic_flow!, N0, tspan, p)   # create problem (system of diff eqs)
#sol = solve(prob)                                     # solve problem

#println(sol)

Conceptual version (doesn't work, need recursive array objects)

In [10]:
# Conceptual version, doesn't work (need recursive array objects)
function traffic_flow!(dN, N, p, t)
    #=
        Returns (change in place)
            - dN: array-like, size (6,)
                    population values at next time step
        Arguments
            - dN: array-like, size (6,)
                    population values at next time step
            - N: array-like, size (6,)
                    population values at current time step
                    N = [P¹, P², C¹₁₂, C¹₂₁, C²₁₂, C²₂₁]
            - p: array-like, size (6,)
                    parameters
                    p = [α₁, α₂, β₁, β₂, p1_out, p2_out]
    =#

    P = N[1]       # array for populations of patches
    C = N[2]       # array for populations in corridors, heading from patch i to patch j

    display(P)
    display(C)

    n = length(P)  # total number of patches
    K = length(C)  # maximum number of connective corridors between patches. Start with K=1.

    αᵢ = p[1]       # tolerance for congestion, originators in patch i
    βₖ = p[2]       # inverse road capacity, corridor k, heading from patch i to patch j
    pi_out = p[3]   # overall demand for leaching patch i

    # Fluxes
    Fₚᶜᵏ = Array{Float64}(undef, n, n) # empty array to hold fluxes from patch i into corridor k, heading towards patch j
    Fₖᵖ = Array{Float64}(undef, n, n)  # empty array to hold fluxes from corridor k into patch j (with an origin of patch i)

    for i in 1:n        # from patch i
        for j in 1:n    # to patch j
                Fₚᶜᵏ[i,j] = [pi_out[i] .* exp(-βₖ .* αᵢ[i] .* C[i,j]) .* P[i]] # flux from patch i into corridor k heading in the j direction
                Fₖᵖ[i,j] = exp(-βₖ .* C[i,j]) .* C[i,j]                # flux from corridor k (heading from patch i to patch j) into patch j
        end
    end
    
    # Net changes to state variables
    for i in 1:n
        for j in 1:n
                dN[1,i] = Fₖᵖ[:,i] .- Fₚᶜᵏ[i] # change in population in patch i
                dN[2,i,j] = Fₚᶜᵏ[i,j] .- Fₖᵖ[i,j] # change in population in corridor k going from patch i to patch j
        end
    end
end

# Solve diff eq
P0 = [1,0]
C0 = [[0,0],
     [0,0]]
N0 = [P0, C0]

α = [1,1]
βₖ = [60, 10]
pi_out = [1, 0]
p = [α, βₖ, pi_out]

tspan = (0.0, 100.0)                                  # set time span
prob = ODEProblem(traffic_flow!, N0, tspan, p)   # create problem (system of diff eqs)
sol = solve(prob)                                     # solve problem

println(sol)

LoadError: Non-Number element type inside of an `Array` detected.
Arrays with non-number element types, such as
`Array{Array{Float64}}`, are not supported by the
solvers.

If you are trying to use an array of arrays structure,
look at the tools in RecursiveArrayTools.jl. For example:

If this was a mistake, promote the element types to be
all the same. If this was intentional, for example,
using Unitful.jl with different unit values, then use
an array type which has fast broadcast support for
heterogeneous values such as the ArrayPartition
from RecursiveArrayTools.jl. For example:

```julia
using RecursiveArrayTools
u0 = ArrayPartition([1.0,2.0],[3.0,4.0])
u0 = VectorOfArray([1.0,2.0],[3.0,4.0])
```

are both initial conditions which would be compatible with
the solvers. Or use ComponentArrays.jl for more complex
nested structures.

Element type:
Vector

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evalua

### Practice with RecursiveArrayTools

In [84]:
using RecursiveArrayTools
a = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]
b = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
vA = VectorOfArray(a)
vB = VectorOfArray(b)

v = vA .* vB # Now all standard array stuff works!
println(v)

aa = (rand(5), rand(5))
bb = (repeat([1],5), rand(5))
cc = (repeat([1],5), repeat([1],5))
pA = ArrayPartition(aa)
pB = ArrayPartition(bb)
pC = ArrayPartition(cc)

p = pA .* pB .* pC # Now all standard array stuff works!

println("\n", pA)
println(pB)
println(pC, "\n")
println(p)

# Mix types?
d = [(0.25, 0.5, 0.75), (1.25, 1.5, 1.75), (2.25, 2.5, 2.75)]
pD = ArrayPartition(d)

println("\n", vA)
display(vA)
println(pD)
display(pD)

q = VectorOfArray(a)

for i in 1:3
    for j in 1:3
        println("vA[$j,$i]: ", vA[j,i], ", pD[$i][$j]: ", pD[i][j])
        q[j,i] = vA[j,i] * pD[i][j]
    end
end

#q = vA .* pD
display(q)

VectorOfArray{Float64, 2, Vector{Vector{Float64}}}([[1.0, 4.0, 9.0], [16.0, 25.0, 36.0], [49.0, 64.0, 81.0]])

ArrayPartition{Float64, Tuple{Vector{Float64}, Vector{Float64}}}(([0.0346762937621139, 0.817049280916409, 0.9490547432637231, 0.43544076794479714, 0.09516575780566006], [0.01356005469750765, 0.8021571080849534, 0.5360336850921074, 0.5781890924069595, 0.4882298174062426]))
ArrayPartition{Float64, Tuple{Vector{Int64}, Vector{Float64}}}(([1, 1, 1, 1, 1], [0.8050239747448179, 0.4353354100754614, 0.6481100173471048, 0.4374498539804247, 0.808866781002247]))
ArrayPartition{Int64, Tuple{Vector{Int64}, Vector{Int64}}}(([1, 1, 1, 1, 1], [1, 1, 1, 1, 1]))

ArrayPartition{Float64, Tuple{Vector{Float64}, Vector{Float64}}}(([0.0346762937621139, 0.817049280916409, 0.9490547432637231, 0.43544076794479714, 0.09516575780566006], [0.010916169130344749, 0.3492073935931094, 0.3474088009436782, 0.2529287340464987, 0.39491288079470227]))

VectorOfArray{Float64, 2, Vector{Vector{Float64}}}([[1.0, 2.0

VectorOfArray{Float64,2}:
3-element Vector{Vector{Float64}}:
 [1.0, 2.0, 3.0]
 [4.0, 5.0, 6.0]
 [7.0, 8.0, 9.0]

ArrayPartition{Float64, Tuple{Vector{Tuple{Float64, Float64, Float64}}}}(([(0.25, 0.5, 0.75), (1.25, 1.5, 1.75), (2.25, 2.5, 2.75)],))


([(0.25, 0.5, 0.75), (1.25, 1.5, 1.75), (2.25, 2.5, 2.75)],)

vA[1,1]: 1.0, pD[1][1]: 0.25
vA[2,1]: 2.0, pD[1][2]: 0.5
vA[3,1]: 3.0, pD[1][3]: 0.75
vA[1,2]: 4.0, pD[2][1]: 1.25
vA[2,2]: 5.0, pD[2][2]: 1.5
vA[3,2]: 6.0, pD[2][3]: 1.75
vA[1,3]: 7.0, pD[3][1]: 2.25
vA[2,3]: 8.0, pD[3][2]: 2.5
vA[3,3]: 9.0, pD[3][3]: 2.75


VectorOfArray{Float64,2}:
3-element Vector{Vector{Float64}}:
 [0.25, 1.0, 2.25]
 [5.0, 7.5, 10.5]
 [15.75, 20.0, 24.75]

In [72]:
#d = [[0.25, 0.5, 0.75], [1.25, 1.5, 1.75], [2.25, 2.5, 2.75]]
#d = ((0.25, 0.5, 0.75), (1.25, 1.5, 1.75), (2.25, 2.5, 2.75))
d = [(0.25, 0.5, 0.75), (1.25, 1.5, 1.75), (2.25, 2.5, 2.75)]
pD = ArrayPartition(d)

i = 2
j = 3

println("EXPERIMENT")
println(pD[i])
println(pD[i][j])
println(pD.[i,j])

EXPERIMENT
(1.25, 1.5, 1.75)
1.75


LoadError: syntax: invalid syntax "pD.[i, j]" around In[72]:12

In [134]:
y = [1,2,3]
z = [4,5]
A = ArrayPartition(y, z)
println(z)
A.x[2]

[4, 5]


2-element Vector{Int64}:
 4
 5

In [131]:
n=2
#empty_matrix = Matrix{Float64}(undef, n, n)
#empty_matrix = Matrix([[1,2] [3,4]])
empty_arr = [0, 0.5]
empty_matrix = [1 2; 3 4]
display(empty_matrix)
println(empty_matrix[:,1])

F = VectorOfArray([empty_arr, empty_matrix])
display(F)
println(F[2][3])

2×2 Matrix{Int64}:
 1  2
 3  4

[1, 3]


VectorOfArray{Float64,2}:
2-element Vector{Union{Matrix{Int64}, Vector{Float64}}}:
 [0.0, 0.5]
 [1 2; 3 4]

2


## Brainstorming matrix representation
P = [P1, P2, ... Pi]

Connections

number of connections across patches, assume unidirectional, so this matrix doesn't necessarily have to be symmetric about the diagonal
P_conn = 

        [[1, 0, ... 1],

          [0, 2, ... 1],

          ...
          
          [2, 0, ... 1]]

Notice each of these are already directional. Also if there is only one corridor per connected pair of patches, we just need C1. For more, will need multiple matrices Cj.
Ck = 

    [[0, C12, ... C1i],
     
     [C21, 0, ... C2i],
     
     ...
     
     [Ci1, Ci2, ... 0]]

dCk[i,j] = Flow_from_Pi_into_Ck - Flow_from_Ck_into_Pj

Note that not all these patches and corridors are connected, but that's okay -- their flux will just always be zero. Also, this matrix might be three dimensional if there are multiple corridors connecting the same two patches!

i is start, j is final destination, k is corridor (does that make sense if there are jointed corridors?)

F_Pi_through_Ck_heading_to_Pj = [[Flux(P1,P1), Flux(P1,P2), ... 

Flux(P1,Pj)],
                         [Flux(P2,P1), Flux(P2,P2), ... Flux(P2,Pj)],
                         ...
                         [Flux(Pi,P1), Flux(Pi,P2), ... Flux(Pi,Pj)]]

F_Pi_through_Ck_heading_to_Pj[i,j] = exp(-alpha * beta * Ck[i,j]) * P[i]

What is the largest number of connections that we might have for a single node? Probably not more than like 10, right? So it's probably okay to have separate matrices for each Ck.

In [4]:
Flow_from_Pi_into_C1_heading_to_Pj[i,j] = exp(-C1[i,j])*P[i]
Flow_into_Pj_from_C1_starting_at_Pi[i,j] = exp(-C1[i,j])*C1[i,j]
dC1[i,j] = Flow_from_Pi_into_C1_heading_to_Pj[i,j] - Flow_into_Pj_from_C1_starting_at_Pi[i,j]
dP[i] = Flow_into_Pj_from_C1_starting_at_Pi^T

LoadError: UndefVarError: `C1` not defined

In [5]:
P_conn = [0 1; 
          1 0]
P = [0.3; 0.3]
C1 = [0 0.3;
      0.1 0]
Flow_Pi_into_C1_heading_to_Pj = exp(-C1)*P
Flow_Pj_out_of_C1_starting_from_Pi = exp(-C1)*C1
dC1 = Flow_Pi_through_C1_to_Pj
display(P_conn)

LoadError: UndefVarError: `Flow_Pi_through_C1_to_Pj` not defined

In [ ]:
P = [0.3; 0.3]
display(P)

2-element Vector{Float64}:
 0.3
 0.3

In [ ]:
P = [0.3; 0.3]
C1 = [0 0.3;
      0.1 0]
C1*P

2-element Vector{Float64}:
 0.09
 0.03

Attempting matrix representation for simple case

In [6]:
function traffic_flow!(dN, N, p, t)
    P¹ = N[1]      # population in patch 1
    P² = N[2]      # population in patch 2
    C¹₁₂ = N[3]    # population in corridor 1, lane from patch 1 to patch 2
    C¹₂₁ = N[4]    # population in corridor 1, lane from patch 2 to patch 1
    C²₁₂ = N[5]    # population in corridor 2, lane from patch 1 to patch 2
    C²₂₁ = N[6]    # population in corridor 2, lane from patch 2 to patch 1

    α₁ = p[1]       # tolerance for congestion, originators in patch 1
    α₂ = p[2]       # tolerance for congestion, originators in patch 2
    β₁ = p[3]       # inverse road capacity, corridor 1
    β₂ = p[4]       # inverse road capacity, corridor 2
    p1_out = p[5]   # overall demand for leaving P1
    p2_out = p[6]   # overall demand for leaving P2

    for i in 1:n
        for j in 1:n
            Fₚᶜᵏ[i,j] = [p1_out * exp(-β₁ * α₁ * Cᵏ[i,j]) * P[i]] # flux from patch i into corridor k heading in the j direction
            Fᵪₖᵖ = exp(-β₁ * Cᵏ[i,j]) * C[i,j]                    # flux from corridor k (heading from patch i to patch j) into patch j
        end
    end

    # Fluxes
    F₁ᶜ¹ = p1_out * exp(-β₁ * α₁ * C¹₁₂) * P¹   # flux from patch 1 into corridor 1 (implicitly heading towards patch 2)
    F₂ᶜ¹ = p2_out * exp(-β₁ * α₂ * C¹₂₁) * P²   # flux from patch 2 into corridor 1 (implicitly heading towards patch 1)
    F₁ᶜ² = p1_out * exp(-β₂ * α₁ * C²₁₂) * P¹   # flux from patch 1 into corridor 2 (implicitly heading towards patch 2)
    F₂ᶜ² = p2_out * exp(-β₂ * α₂ * C²₂₁) * P²   # flux from patch 2 into corridor 2 (implicitly heading towards patch 1)

    Fc₁¹ = exp(-β₁ * C¹₂₁) * C¹₂₁     # flux from corridor 1 into patch 1 (implicitly lane from p2 to p1)
    Fc₁² = exp(-β₁ * C¹₁₂) * C¹₁₂     # flux from corridor 1 into patch 2 (implicitly lane from p1 to p2)
    Fc₂¹ = exp(-β₂ * C²₂₁) * C²₂₁     # flux from corridor 2 into patch 1 (implicitly lane from p2 to p1)
    Fc₂² = exp(-β₂ * C²₁₂) * C²₁₂     # flux from corridor 2 into patch 2 (implicitly lane from p1 to p2)
    
    # Net changes to state variables
    dN[1] = Fc₁¹ + Fc₂¹ - F₁ᶜ¹ - F₁ᶜ² # change in population Patch 1
    dN[2] = Fc₁² + Fc₂² - F₂ᶜ¹ - F₂ᶜ² # change in population Patch 2
    dN[3] = F₁ᶜ¹ - Fc₁² # change in population Corridor 1 lane from p1 to p2
    dN[4] = F₂ᶜ¹ - Fc₁¹ # change in population Corridor 1 lane from p2 to p1
    dN[5] = F₁ᶜ² - Fc₂² # change in population Corridor 2 lane from p1 to p2
    dN[6] = F₂ᶜ² - Fc₂¹ # change in population Corridor 2 lane from p2 to p1
end
# Where to put conservation law??

pred! (generic function with 1 method)

In [7]:
# Choose parameters
α₁ = 1
α₂ = 1
β₁ = 10
β₂ = 60
p1_out = 1
p2_out = 0
p = [α₁, α₂, β₁, β₂, p1_out, p2_out]

P0 = [0.9, 0.1] # initial population in each patch
C0 = [0 0; 0 0] # assume max 1 corridor for each connection, and no connecting the same og and dest

# Solve diff eq
using DifferentialEquations
N0 = [1; 0.; 0; 0; 0; 0]
tspan = (0.0, 100.0)
prob = ODEProblem(pred!, N0, tspan, p)
sol = solve(prob)
using Plots
#plot(sol, idxs = (0, 1, 2))
plot(sol, idxs = (0,1), xlabel="time", ylabel="population", label="patch 1")
plot!(sol, idxs = (0,2), xlabel="time", ylabel="population", label="patch 2")
plot!(sol, idxs = (0,3), xlabel="time", ylabel="population", label="C1 (1 to 2), β₁=$β₁")
plot!(sol, idxs = (0,5), xlabel="time", ylabel="population", label="C2 (1 to 2), β₂=$β₂")
plot!(sol, idxs = (0,4), xlabel="time", ylabel="population", label="C1 (2 to 1), β₁=$β₁")
plot!(sol, idxs = (0,6), xlabel="time", ylabel="population", label="C2 (2 to 1), β₂=$β₂")

LoadError: UndefVarError: `n` not defined